## Tobigs 9주차 모델 심화2
#### 15기 이윤정
---   

# `[Attention is all you need] Paper Review`
* 논문 저자 : Google
* 논문 링크 : https://arxiv.org/pdf/1706.03762.pdf
---
   ## 0. Abstract
   * **Transfomer : 다른 CNN(convolution) 혹은 RNN(recursive) 모델 없이 단순히 Attention Mechanism을 통해 구성된 모델**
   * 기존의 모델보다 성능이 좋고, 병렬화를 통해 연산량은 감소한 모델

## 1. Introduction
   * RNN 모델의 단점 : sequence가 길어질 수록 성능이 하락 + momory의 제약(batch에 제한)
   * Sequence2Sequence 모델의 단점 : sequence의 alignment 해결 X
   * 해당 모델들의 단점을 보완하기 위해 Attention이라는 개념이 도입되었으나 대부분 RNN과 함께 사용되는 선에서 그침
   * 다른 모델 없이 attention만으로 구성되어 병렬화가 가능한 Transfomer 모델의 등장   

## 2. Background
* 기존에 행해진 sequential 계산을 줄이려는 노력은 모두 CNN을 기본 block으로 사용하므로 input-output 간의 관련성을 파악하기 위한 연산량은 거리에 따라 증가한다. (ConvS2s - linear, ByteNet - logarithmically) 
* 따라서, 장거리 dependency 학습에 어려움이 존재한다. ( = 거리가 멀수록 dependency 학습에 어려움)  
<br>
* **반면에, Transfomer 모델은 상수 시간의 계산으로 연산이 가능하다.**
* 단점 : attention weighted position을 평균냄 = 해상도 감소
* 보완 : **Multi-Head Attention**으로 상쇄 가능  
<br>
* **Self-Attention(Intra-Attention)**   
: 1개의 sequence에 대한 representation을 계산하기 위해 동일한 sequence(input, output ... ) 내의 원소들 사이에서 attention을 수행하는 매커니즘<br><br>
    즉, 자기 자신의 attention을 계산하는 매커니즘으로 특정 feature(it - 대명사)가 같은 context 속 어떤 features(animal - 대명사 지칭하는 것)를 참고하고 있는 지 attention을 계산하는 것
    
    ![](http://jalammar.github.io/images/t/transformer_self-attention_visualization.png)

## 3. Model Architecture
* 기존의 Sequence Model : encoder-decoder 구조
    * encoder : input sequence > continuous representation
    * decoder : continuous representation > output 
* Transformer Model : Stacked Self-Attention + Point-Wise Fully Connected Layer 구조 (encoder와 decoder 여러 개를 중첩한 구조)

![](http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png)

* encoder : input sequence ($x_1, ..., x_n$) > $z = (z_1, ..., z_n)$으로 mapping
    - 총 6개의 층으로 각 층은 2개의 sublayer로 구성되어 있다. 
    - 첫번째 sublayer : multi-head self-attention layer - 한 토큰이 다른 토큰과 얼마나 관련되어 있는 지 가중치 계산
    - 두번째 sublayer : position-wise fully connected feed-forward network
<br><br>      
* decoder : output sequence = $y=(y_1, ..., y_n)$
    - 총 6개의 층으로 각 층은 3개의 sublayer로 구성되어 있다. 
    - 첫번째 sublayer : (Masked) multi-head self-attention layer - 미래의 위치에 접근을 불가하고 해당 위치와 그 이전 위치에 대해서만 의존하게 만드는 **masking 기법** 사용 (=지금까지 출력된 단어에 대해서만 attention을 적용)
    - 두번째 sublayer : multi-head self-attention layer
    - 세번째 sublayer : position-wise fully connected feed-forward network

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FnggqX%2FbtqBS6JNjRN%2F5JFBAKaANPoJGJtFBZ45L0%2Fimg.gif)


### Attention
* Query + Key-Value 쌍의 집합을 output에 mapping 하는 것

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FyEx9z%2FbtqOc92LjUh%2Fy5qKmo1kOGDiF1qij5JpE1%2Fimg.png)


#### Scaled Dot-Product Attention
1. 각 input vector로 부터 세 벡터 (Queries, Keys, Values)를 만든다.  
    : $(trainable weights) * (input word의 임베딩 x)$  <br>
2. 특정 위치의 단어가 다른 단어와 얼마나 연관되어 있는 지 계산  
    : Query와 Key vector 간의 dot product  <br>
3. 계산된 점수를 Key vector의 차원 수의 제곱근으로 나눈 후 softmax  
    : $Softmax(\frac{QK^T}{\sqrt{d_k}})$  <br>
4. Value Vector에 softmax score를 곱한다.  
    : $Attention(Q, K, V) = Softmax(\frac{QK^T}{\sqrt{d_k}})*V$
    
#### Multi-Head Attention
Queries, Keys, Values를 통한 단일 Attention을 수행하는 것보다 $d_{model}$의 차원의 벡터를 h개로 나눠서 각각 $d_q, d_k, d_v$크기의 벡터들을 병렬로 하는 하는 방법 (일종의 앙상블)  
각 연산은 결과적으로 $d_v$ 차원의 벡터를 제공하는 데 모든 값을 다 구하면 다시 합쳐 input으로 제공한 $d_{model}$ 차원의 벡터로 만들어 FC layer의 input으로 제공된다.

#### Position-wise Feed-Forward Network
모든 encoder, decoder 층에 FFN 층이 존재한다. P이는 ReLU 함수를 activation function으로 사용하는 2번의 linear transformation으로 구성된다.

#### Positional Encoding
Transformer Model은 Convilution도 Recurrence도 사용하지 않으므로 순서에 따른 Sequence의 위치 정보를 주입해주어야하낟. 이를 위해 encoder 및 decoder stack 하단에 Positional encoding을 추가한다. 이는 embedding과 동일한 차원 $d_(model)$을 갖기 때문에 더하는 작업을 수행할 수 있다.

## 4. Why Self-Attention
#### Self-Attention의 장점
1. 각 Layer에서 발생하는 계산 복잡도의 감소
2. 병렬화하여 연산 가능
3. 멀리 떨어진 원소 간의 path length 감소 = 장거리 학습의 가능
4. model 자체의 동작을 해석하기 원활함

![](https://media.vlpt.us/images/changdaeoh/post/5fc3dd1b-a466-4fa5-8f9b-0717bce77573/image.png)

자연어 처리에서는 input으로 사용되는 sequence의 길이 N이 embeding의 차원 d보다 대부분 작다. 따라서, convolution이나 resurrence보다 self-attention의 Layer에서 발생하는 계산복잡도가 작다. 추가로 의존성을 가질 있는 maximum path len 역시 self-attention은 문장 내 모든 단어에 1번씩만 접근하므로 convolution이나 resurrence보다 self-attention의 계산복잡도가 작다.

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FwFerG%2FbtqOcRusXwE%2FslN0ENq1drhXyvj2K05DJ1%2Fimg.png)

## 5. Training
|    Parameter    |                                                    Descrption                                                   |
|:---------------:|:---------------------------------------------------------------------------------------------------------------:|
| DataSet(German) |                           WMT 2014 English-German dataset (4.5M쌍의 문장, 37000 vocab)                          |
| DataSet(French) |                           WMT 2014 English-French dataset (36M쌍의 문장, 32000 vocab)                           |
|    Batch size   |                                                      25000                                                      |
|     Hardware    |                                                  8개의 P100 GPU                                                 |
|    Optimizer    |                                                       Adam                                                      |
|  Learning Rate  | ![image](https://user-images.githubusercontent.com/69336270/112943505-85125a00-916c-11eb-97f5-27ef174fe175.png) |
|   warmup_steps  |                                                       4000                                                      |
|  Regularization |                                                 Residual Dropout                                                |

## 6. Results
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb1x23m%2FbtqN7TNTXhA%2FPMyM7tmJM6g0huKaxmE6Qk%2Fimg.png)

Transfomer Model이 이전의 Model과 비교하였을 때, 속도와 성능의 면에서 모두 우수하다는 것을 확인할 수 있다.

## 7. Conclusion
번역에서 Tansformer Model은 Recurrence 및 convolution을 사용하지 않고, Attention만 사용해서 만든 모델임에도 이전의 Model과 비교하였을 때 우수한 성능을 가지고 있는 것을 입증하였다. 앞으로 Transfomer model은 번역 이외에도 이미지, 오디오 등 큰 input을 가진 data에도 적용할 수 있을 것이다.

## 8. Reference
[1] https://hipgyung.tistory.com/12  
[2] https://reniew.github.io/43/  
[3] https://greeksharifa.github.io/nlp(natural%20language%20processing)%20/%20rnns/2019/08/17/Attention-Is-All-You-Need/#%EC%B4%88%EB%A1%9Dabstract